# LangGraph

LangGraph is a framework providing a durable runtime for AI agents and applications.

## Key benefits of LangGraph

1. **Latency**: LLMs have high latency
   - Parallel execution of tasks (reduces latency by factor of N)
   - Streaming support for immediate token delivery (optimizes time to first token)
2. **Reliability**: Long-running agents can fail, causing expensive reruns
   - Checkpointing: saves application state at each step
   - Allows resumption from exact failure point
3. **Non-deterministic Responses**: LLMs produce variable results across calls and models
   - Human-in-the-loop interface
   - Tracing and evaluation via LangSmith

## Prerequisite

- OpenAI API key:
  - https://platform.openai.com/api-keys
- LangSmith API key:
  - https://docs.langchain.com/langsmith/create-account-api-key

## Components in LangGraph

Component | Description
----------|------------
State     | **Data** that flows through the graph
Node      | **Functions** that operate on the data
Edges     | Control Flow
Checkpointing | State persistence across time and failures
Control Flow  | Can be interrupted and resumed (enables human-in-the-loop)

### State

- Data supplied to, updated by, and returned from the graph
- **Graphs are stateless** - state flows through them
- Shared by all nodes
- Define as: TypedDict, Python dataclass, or Pydantic BaseModel

### Nodes

- Simply **functions**:
  - `node(state) -> updated_state`
- Receive current state as input
- Return updates to state
- Can access and modify shared state

### Edges

- Define control flow between nodes
- Types: static or conditional
- Execution: parallel or series

### Execution Flow

1. Initialise state when graph is invoked
2. Runtime selects and executes nodes
3. Nodes receive current state and return updates
4. State is updated with node results
5. Final state is returned on completion

### State Persistence

- State can be persisted across time and failures
- If node fails → restart and restore state
- Function reruns from beginning
- Critical for building resilient applications

## Basic Pipeline

1. Setup:
   - **State**: All nodes share the same state, which can be `TypedDict`, `dataclass` or `BaseModel`
   - **Nodes**: Defined as simple python functions that receive state as ainput and return updated state
2. Execution:
   - Runtime: When you call `invoke`, the graph initialises the input state from the invoke statement and determines which nodes to run
   - State Flow: Each node receives the current state as input, executes its logic, and returns an updated state
   - Graph Return: After all nodes complete execution, the graph returns the final state value

```mermaid
---
config:
  flowchart:
    curve: linear
---
graph LR;
	__start__([<p>__start__</p>]):::first
	a(a)
	__end__([<p>__end__</p>]):::last
	__start__ --> a;
	a --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc
```

In [11]:
from typing import TypedDict
from langgraph.graph import END, START, StateGraph

############################################################
# Setup
class State(TypedDict):
    nlist: list[str]

def node_a(state: State) -> State:
    print(f"node a is receiving {state["nlist"]}")
    note = "Hello world from Node a"
    return State(nlist = [note])

builder = StateGraph(State)
builder.add_node("a", node_a)
builder.add_edge(START, "a")
builder.add_edge("a", END)
graph = builder.compile()
# print(graph.get_graph().draw_mermaid())

############################################################
# Execution
initial_state = State(
    nlist = ["Hello node a"]
)
result = graph.invoke(initial_state)
print(result)

node a is receiving ['Hello node a']
{'nlist': ['Hello world from Node a']}


## Edge

### Edge Types

- **Static Edges** (solid lines)
  - Always taken
  - Serial execution: Node runs after parent completes
  - Parallel execution: Multiple nodes run simultaneously via `add_edge()` from same parent
- **Conditional Edges** (dashed lines)
  - Evaluated at runtime
  - Enable branching logic based on conditions
  - Only taken paths execute

### Super Steps

- Execution unit where all active nodes must complete before continuing
- All node outputs stored to state before next super step begins

### Reducer Functions

Reducer provide custom behaviour to update the state.
- For example `operator.add` can merges values of each node.
- Custom reducers supported

```python
class State(TypedDict):
    nlist: Annotated[list[str], operator.add]
```

### Execution Behavior

Edges define control flow, NOT data access
- Nodes see full current graph state (including parallel branch updates)
- Nodes in same super step see identical input state

```mermaid
---
config:
  flowchart:
    curve: linear
---
graph LR;
	__start__([<p>__start__</p>]):::first
	a(a)
	b(b)
	c(c)
	bb(bb)
	cc(cc)
	d(d)
	__end__([<p>__end__</p>]):::last
	__start__ --> a;
	a --> b;
	a --> c;
	b --> bb;
	bb --> d;
	c --> cc;
	cc --> d;
	d --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc
```

In [3]:
from typing import TypedDict, Annotated
import operator

class State(TypedDict):
    nlist: Annotated[list[str], operator.add]

def node_a(state: State) -> State:
    print(f"Adding 'A' to {state['nlist']}")
    return(State(nlist = ["A"]))

def node_b(state: State) -> State:
    print(f"Adding 'B' to {state['nlist']}")
    return(State(nlist = ["B"]))

def node_c(state: State) -> State:
    print(f"Adding 'C' to {state['nlist']}")
    return(State(nlist = ["C"]))

def node_bb(state: State) -> State:
    print(f"Adding 'BB' to {state['nlist']}")
    return(State(nlist = ["BB"]))

def node_cc(state: State) -> State:
    print(f"Adding 'CC' to {state['nlist']}")
    return(State(nlist = ["CC"]))

def node_d(state: State) -> State:
    print(f"Adding 'D' to {state['nlist']}")
    return(State(nlist = ["D"]))

In [10]:
from langgraph.graph import StateGraph

builder = StateGraph(State)

# Add nodes
builder.add_node("a", node_a)
builder.add_node("b", node_b)
builder.add_node("c", node_c)
builder.add_node("bb", node_bb)
builder.add_node("cc", node_cc)
builder.add_node("d", node_d)

# Add edges
builder.add_edge(START,"a")
builder.add_edge("a", "b")
builder.add_edge("a", "c")
builder.add_edge("b", "bb")
builder.add_edge("c", "cc")
builder.add_edge("bb", "d")
builder.add_edge("cc", "d")
builder.add_edge("d",END)

# Compile and display
graph = builder.compile()
# print(graph.get_graph().draw_mermaid())

In [9]:
initial_state = State(
    nlist = ["Initial String:"]
)
graph.invoke(initial_state)

Adding 'A' to ['Initial String:']
Adding 'B' to ['Initial String:', 'A']
Adding 'C' to ['Initial String:', 'A']
Adding 'BB' to ['Initial String:', 'A', 'B', 'C']
Adding 'CC' to ['Initial String:', 'A', 'B', 'C']
Adding 'D' to ['Initial String:', 'A', 'B', 'C', 'BB', 'CC']


{'nlist': ['Initial String:', 'A', 'B', 'C', 'BB', 'CC', 'D']}

## Conditional Edges

Conditional edges enable **dynamic routing** where the next node to execute depends on the current state at runtime.

```mermaid
---
config:
  flowchart:
    curve: linear
---
graph LR;
	__start__([<p>__start__</p>]):::first
	a(a)
	b(b)
	c(c)
	__end__([<p>__end__</p>]):::last
	__start__ --> a;
	a -.-> __end__;
	a -.-> b;
	a -.-> c;
	b --> __end__;
	c --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc
```

In [12]:
from typing import TypedDict, Annotated
import operator

class State(TypedDict):
    nlist : Annotated[list[str], operator.add]

### Method 1: `add_conditional_edges` with a Routing Function

This approach **separates routing logic from node logic**.

Define a separate function that returns the next node based on state.

In [15]:
from typing import Literal
from langgraph.graph import StateGraph, END, START

def node_a(state: State):
    return
def node_b(state: State) -> State:
    return(State(nlist = ["B"]))
def node_c(state: State) -> State:
    return(State(nlist = ["C"]))

def conditional_edge(state: State) -> Literal["b", "c", END]:
    select = state["nlist"][-1]
    if select == "b":
        return "b"
    elif select == "c":
        return "c"
    elif select == "q":
        return END
    else:
        return END

builder = StateGraph(State)
builder.add_node("a", node_a)
builder.add_node("b", node_b)
builder.add_node("c", node_c)

builder.add_edge(START, "a")
builder.add_conditional_edges("a", conditional_edge)

graph = builder.compile()
# print(graph.get_graph().draw_mermaid())


### Method 2: `Command` in Node Return

Bundle the routing decision directly in the node:

```python
return Command(
    update = State(nlist = [select]),
    goto = [next_node]
)
```

- `Command` allows to:
  - **`update`**: Update the state
  - **`goto`**: Specify the next node(s) to execute
- **Type annotations** on `Command` return types don't affect execution but enable accurate graph visualisation
- **`goto` can be a list** of nodes for multiple paths
- **Routing decisions are checked at runtime**, so node names must match exactly

In [ ]:
from IPython.display import Image
from langgraph.graph import END
from langgraph.types import Command
def node_a(state: State) -> Command[Literal["b", "c", END]]:
    select = state["nlist"][-1]
    if select == "b":
        next_node = "b"
    elif select == "c":
        next_node = "c"
    elif select == "q":
        next_node = END
    else:
        next_node = END
    return Command(
        update = State(nlist = [select]),
        goto = [next_node]
    )
def node_b(state: State) -> State:
    return(State(nlist = ["B"]))
def node_c(state: State) -> State:
    return(State(nlist = ["C"]))

builder = StateGraph(State)

builder.add_node("a", node_a)
builder.add_node("b", node_b)
builder.add_node("c", node_c)

builder.add_edge(START, "a")
builder.add_edge("b", END)
builder.add_edge("c", END)

graph = builder.compile()
# print(graph.get_graph().draw_mermaid())


## Memory and Persistence

**Checkpointer**
- Stores state snapshots at the end of each super step
- Gives graph persistent memory across runs
- Enables failure recovery and state rollback

**Thread**
- Collection of checkpoints over time
- Identified by `thread_id`
- Same `thread_id` → shared state history
- Different `thread_id` → separate conversation histories

### Benefits of Checkpointers

1. **Failure recovery**: Restore state and resume without losing progress
2. **Time-travel**: Roll back to earlier checkpoint when needed
3. **Persistence**: State survives beyond graph execution
4. **Interrupts**: Resume execution from exact suspension point

### Implementations

- **`InMemorySaver`**: In-memory storage
- **`PostgresSaver`**: PostgreSQL database
- **`SQLiteSaver`**: SQLite database

### Usage Pattern

```python
from langgraph.checkpoint.memory import InMemorySaver

memory = InMemorySaver()
config = {"configurable": {"thread_id": "1"}}
graph = builder.compile(checkpointer=memory)
result = graph.invoke(input_state, config)
```

- Without checkpointer: Each invocation starts fresh
- With checkpointer: State accumulates within same thread across runs

In [25]:
from langgraph.checkpoint.memory import InMemorySaver
memory = InMemorySaver()
config = {"configurable": {"thread_id": "1"}}
graph = builder.compile(checkpointer=memory)

command_sequence = ['b', 'c', 'q']
for command in command_sequence:
    input_state = State(nlist = [command])
    result = graph.invoke(input_state, config)
    print( result )
    if result['nlist'][-1] == "q":
        print("quit")
        break

{'nlist': ['b', 'b', 'b2', 'B']}
{'nlist': ['b', 'b', 'b2', 'B', 'c', 'c', 'c2', 'C']}
{'nlist': ['b', 'b', 'b2', 'B', 'c', 'c', 'c2', 'C', 'q', 'q', 'q2']}


## Interrupts

`interrupt()` pauses graph execution to wait for external input, then resumes exactly where it stopped.

**Requirements**: Must use a checkpointer to persist state between pause and resume.

### Benefits of Interrupts

- **Human approval** before sensitive operations
- **Admin review** for unexpected situations
- **User input** to resolve ambiguities

### Basic Pattern

**Raise interrupt in node:**
```python
from langgraph.types import interrupt

def node_a(state: State):
    # ... logic ...
    user_input = interrupt(f"Need approval for: {action}")
    # Code continues after resume with user_input value
    return Command(update=state, goto=next_node)
```

**Handle interrupt and resume:**
```python
result = graph.invoke(input_state, config)

if '__interrupt__' in result:
    msg = result['__interrupt__'][-1].value
    human_response = input(msg)

    # Resume with same thread_id
    result = graph.invoke(Command(resume=human_response), config)
```

### Execution Flow

1. `interrupt()` called -> graph pauses
2. State checkpointed
3. Interrupt message returned in `result['__interrupt__']`
4. External input collected (can take seconds to hours)
5. Resume with `Command(resume=value)` using same `thread_id`
6. Node replays from beginning with interrupt value supplied

### Key Behaviors

**Node Replay**
- Nodes restart from the top (not mid-execution) on resume
- Code before `interrupt()` re-executes
- Previously seen interrupts auto-supplied (won't hit same interrupt twice)

**Multiple Interrupts**
- Same node: Can have multiple sequential interrupts
- Parallel nodes: `result['__interrupt__']` is a list

In [26]:
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import Command, interrupt
from langgraph.graph import END

memory = InMemorySaver()
config = {"configurable": {"thread_id": "1"}}

def node_a(state: State) -> Command[Literal["b", "c", END]]:
    print("Entered 'a' node")
    select = state["nlist"][-1]
    if select == "b":
        next_node = "b"
    elif select == "c":
        next_node = "c"
    elif select == "q":
        next_node = END
    else:
        admin = interrupt(f"Unexpected input '{select}'")
        print(admin)
        if admin == "continue":
            next_node = "b"
        else:
            next_node = END
            select = "q"
    return Command(
        update = State(nlist = [select]),
        goto = next_node
    )
def node_b (state: State) -> State:
    return(State(nlist = ["B"]))
def node_c (state: State) -> State:
    return(State(nlist = ["C"]))

builder = StateGraph(State)
builder.add_node("a", node_a)
builder.add_node("b", node_b)
builder.add_node("c", node_c)
builder.add_edge(START,"a")
builder.add_edge("b", END)
builder.add_edge("c", END)
graph = builder.compile(checkpointer=memory)

command_sequence = ['b', 'c', 'q']
for command in command_sequence:
    input_state = State(nlist = [command])
    result = graph.invoke(input_state, config)
    
    if '__interrupt__' in result:
        print(f"Interrupt:{result}")
        msg = result['__interrupt__'][-1].value
        print(msg)
        human = input(f"\n{msg}: ")

        human_response = Command(
            resume = human
        )
        result = graph.invoke(human_response, config)
        
    if result['nlist'][-1] == "q":
        print("quit")
        break

Entered 'a' node
Entered 'a' node
Entered 'a' node
quit
